In [2]:
# automatically reload changes we make to schools.py and ui.py
%load_ext autoreload
%autoreload 2

# load the demographic data
import pandas as pd
from nycschools import class_size, config, schools
data_dir = config.data_dir
import os
beds = schools.load_school_demographics().beds.unique()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
acs = pd.read_json("https://api.census.gov/data/2021/acs/acs1?get=NAME,group(B01001)&for=us:1")
acs.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,NAME,B01001_001E,B01001_001EA,B01001_001M,B01001_001MA,B01001_002E,B01001_002EA,B01001_002M,B01001_002MA,B01001_003E,...,B01001_048EA,B01001_048M,B01001_048MA,B01001_049E,B01001_049EA,B01001_049M,B01001_049MA,GEO_ID,NAME,us
1,United States,331893745,None,-555555555,*****,164350703,None,26594,None,9552445,...,None,26703,None,3791691,None,28666,None,0100000US,United States,1


In [8]:
df = class_size.get_class_size()
# config.urls["class_size"]
df.columns

/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Index(['dbn', 'school_name', 'grade', 'program_type', 'students_n',
       'classes_n', 'avg_class_size', 'min_class_size', 'max_class_size',
       'dept', 'subject', 'ay'],
      dtype='object')

In [6]:
ptr = class_size.load_ptr()
ptr.head()

,DBN,School Name,School Pupil-Teacher Ratio
0,01M015,PS 015 ROBERTO CLEMENTE,7.844036
1,01M020,PS 020 ANNA SILVER,9.060864
2,01M034,PS 034 FRANKLIN D ROOSEVELT,7.089503
3,01M063,THE STAR ACADEMY - PS63,9.620253
4,01M064,PS 064 ROBERT SIMON,8.891320


In [21]:
df, ptr = class_size.get_class_22()
ptr.head()


/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/school-data/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,DBN,School Name,School Pupil-Teacher Ratio
0,01M015,PS 015 ROBERTO CLEMENTE,7.844036
1,01M020,PS 020 ANNA SILVER,9.060864
2,01M034,PS 034 FRANKLIN D ROOSEVELT,7.089503
3,01M063,THE STAR ACADEMY - PS63,9.620253
4,01M064,PS 064 ROBERT SIMON,8.891320


In [23]:
url = "https://data.cityofnewyork.us/resource/sgr7-hhwp.csv?$limit=1000000"
class21 = class_size.get_class_size_year(2021, url)
class21.head()


,dbn,school_name,grade,program_type,students_n,classes_n,avg_class_size,min_class_size,max_class_size,ay
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,13,13,2021
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,2021
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,8,8,2021
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,2021
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,8,8,2021


In [15]:

print(class21.columns)
class21.rename(columns={
    "number_of_students": "students_n", 
    "grade_level": "grade",
    "number_of_classes":"classes_n",
    "minimum_class_size":"min_class_size",
    "maximum_class_size":"max_class_size",
    "average_class_size":"avg_class_size"
    }, inplace=True)
class21.columns.difference(class22.columns)

Index(['dbn', 'school_name', 'grade_level', 'program_type',
       'number_of_students', 'number_of_classes', 'average_class_size',
       'minimum_class_size', 'maximum_class_size'],
      dtype='object')


Index([], dtype='object')

In [17]:
def fix_rows(row):
    if row.min_class_size == "<15":
        row.min_class_size = row.students_n
    if row.max_class_size == "<15":
        row.max_class_size = row.students_n
    if row.grade in ['K', 1, 2, 3, 4, 5,]:
        row.subject = "Elementary"
        row.dept = "Elementary"


    return row

class21.apply(fix_rows, axis=1)

,dbn,school_name,grade,program_type,students_n,classes_n,avg_class_size,min_class_size,max_class_size
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,13,13
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,8,8
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,8,8
...,...,...,...,...,...,...,...,...,...
12440,32K377,PS 377 ALEJANDRINA B DE GAUTIER,K-8 SC,SC 12:1:1,16,2,8.0,<6,12
12441,32K383,JHS 383 PHILIPPA SCHUYLER,K-8 SC,SC 12:1,10,1,10.0,10,10
12442,32K383,JHS 383 PHILIPPA SCHUYLER,K-8 SC,SC 12:1:1,34,3,11.3,10,12
12443,32K384,PS /IS 384 FRANCES E CARTER,K-8 SC,SC 12:1:1,51,6,8.5,6,12


In [4]:
d = os.path.join(data_dir, "avg-class-size.csv") # nysed class size data
data = pd.read_csv(d)
# data.grade.unique()
nysed = data[data.ENTITY_CD.isin(beds)]
nysed.columns = ['beds', 'school_name', 'ay', 'program_type', 'avg_class_size', 'data_reported']
nysed


,beds,school_name,ay,program_type,avg_class_size,data_reported
21605,307500010000,NYC SPECIAL SCHOOLS - DISTRICT 75,2019,Algebra I,1.0,Y
21606,307500010000,NYC SPECIAL SCHOOLS - DISTRICT 75,2019,Biology,2.0,Y
21607,307500010000,NYC SPECIAL SCHOOLS - DISTRICT 75,2019,Earth Science,1.0,Y
21608,307500010000,NYC SPECIAL SCHOOLS - DISTRICT 75,2019,ELA,1.0,Y
21609,307500010000,NYC SPECIAL SCHOOLS - DISTRICT 75,2019,Grade 1,2.0,Y
...,...,...,...,...,...,...
148206,353100861144,BRIDGE PREPARATORY CHARTER SCHOOL,2021,Language Arts (grade 3),24.0,Y
148207,353100861144,BRIDGE PREPARATORY CHARTER SCHOOL,2021,Mathematics (grade 3),24.0,Y
148208,353100861175,HELLENIC CLASSICAL CS -STATEN ISLAND,2021,Grade 1,25.0,Y
148209,353100861175,HELLENIC CLASSICAL CS -STATEN ISLAND,2021,Grade 2,21.0,Y


In [6]:
nysed.program_type.value_counts()

Kindergarten                                   2012
Grade 1                                        2011
Grade 2                                        2000
Grade 5 Math                                   1681
Grade 3 Math                                   1673
Grade 4 Science                                1668
Grade 4 Math                                   1663
Grade 3 ELA                                    1655
Grade 5 ELA                                    1646
Grade 4 ELA                                    1645
Biology                                        1162
Grade 6 Math                                    998
Grade 7 Math                                    982
Grade 6 ELA                                     965
Grade 7 ELA                                     950
Grade 8 ELA                                     933
Grade 8 Math                                    887
Mathematics (grade 5)                           857
Language Arts (grade 5)                         856
Language Art

In [8]:
url = "https://infohub.nyced.org/docs/default-source/default-document-library/updated2023_avg_classsize_schl.xlsx"
xls = pd.read_excel(url, sheet_name=None)
# first sheet is k-8
k8 = xls["K-8 Avg"]
hs = xls["MS HS Avg"]
ptr = xls["PTR"]

df = pd.concat([k8, hs], axis=0)
df["ay"] = 2022

df.columns = ['dbn', 'school_name', 'grade', 'program_type',
              'students_n', 'classes_n', 'avg_class_size',
              'min_class_size', 'max_class_size', 'dept', 'subject',
              'ay']

df.head()

/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,dbn,school_name,grade,program_type,students_n,classes_n,avg_class_size,min_class_size,max_class_size,dept,subject,ay
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,11,1,11.0,<15,<15,NaN,NaN,2022
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,25,1,25.0,25,25,NaN,NaN,2022
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,14,1,14.0,<15,<15,NaN,NaN,2022
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,17,1,17.0,17,17,NaN,NaN,2022
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,7,1,7.0,<15,<15,NaN,NaN,2022


In [10]:


# df = df.apply(lambda x: x["Number of Students"] if x["Min Class Size"] < 15 else x["Min Class Size"], axis=1)


# make Min Class Size the same as Number of Students if <15
def fix_class_size(row):
    if row.min_class_size == "<15":
        row.min_class_size = row.students_n
    if row.max_class_size == "<15":
        row.max_class_size = row.students_n

    return row


df = df.apply(fix_class_size, axis=1)
df.head()




,dbn,school_name,grade,program_type,students_n,classes_n,avg_class_size,min_class_size,max_class_size,dept,subject,ay
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,11,1,11.0,11,11,NaN,NaN,2022
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,25,1,25.0,25,25,NaN,NaN,2022
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,14,1,14.0,14,14,NaN,NaN,2022
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,17,1,17.0,17,17,NaN,NaN,2022
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,7,1,7.0,7,7,NaN,NaN,2022
